In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import xgboost 
import os
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesClassifier,
)
from sklearn.svm import SVR
from scipy.stats import randint, uniform

## Process Weather Data

In [ ]:
weather_data = pd.read_csv("../data/processed_weather_data_leuven.csv", index_col=0)


In [ ]:
weather_data.head(4)

In [ ]:
weather_data.columns

In [ ]:
weather_data.dtypes

In [ ]:
np.unique(weather_data.weathercode)

In [ ]:
# extract from timestamp
weather_data["time"] = pd.to_datetime(weather_data["time"])
weather_data["date"] = weather_data["time"].dt.date
weather_data["hour"] = weather_data["time"].dt.hour
weather_data["month"] = weather_data["time"].dt.month
weather_data["weekday"] = weather_data["time"].dt.strftime("%a")
weather_data.head(5)

In [ ]:
weather_data.drop("time", axis=1, inplace=True)

In [ ]:
weather_data.shape

In [ ]:
weather_data.head(3)

## Process air quality data

In [ ]:
air_quality_data = pd.read_csv("../data/processed_air_quality_data.csv", index_col=0)


In [ ]:
air_quality_data.head(3)

In [ ]:
air_quality_data.columns

In [ ]:
# extract from timestamp
air_quality_data["date"] = pd.to_datetime(air_quality_data["date"]).dt.date
air_quality_data.head(5)



In [ ]:
air_quality_data = (
    air_quality_data.groupby(["date", "hour", "month", "weekday"]).mean().reset_index()
)

In [ ]:
air_quality_data

## Process file42 data

In [ ]:
file42 = pd.read_csv("../data/processed_file42_data.csv", index_col=0)
file42.dropna(inplace=True)
file42.head(5)

In [ ]:
file42.columns

In [ ]:
file42.rename(columns={"#object_id": "object_id"}, inplace=True)

In [ ]:
file42["object_id"] = pd.to_numeric(file42["object_id"]).astype(int)

In [ ]:
file42.head(5)

In [ ]:
# Convert the 'result_timestamp' column to a datetime data type
file42["result_timestamp"] = pd.to_datetime(file42["result_timestamp"])
file42["date"] = file42["result_timestamp"].dt.date
file42["hour"] = file42["result_timestamp"].dt.hour
file42["month"] = file42["result_timestamp"].dt.month
file42["weekday"] = file42["result_timestamp"].dt.strftime("%a")
file42.head(4)

In [ ]:
file42 = (
    file42.groupby(["object_id", "date", "hour", "month", "weekday"])
    .mean()
    .reset_index()
)
file42.head(5)

## Merge data

In [ ]:
data_model = file42.merge(
    air_quality_data,
    how="inner",
    left_on=["date", "hour", "month", "weekday"],
    right_on=["date", "hour", "month", "weekday"],
)

In [ ]:
data_model = data_model.merge(
    weather_data,
    how="inner",
    left_on=["date", "hour", "month", "weekday"],
    right_on=["date", "hour", "month", "weekday"],
)

In [ ]:
data_model.shape

In [ ]:
data_model.head(5)

In [ ]:
data_model.dtypes

In [ ]:
## split train, test data
train_df, val_df = train_test_split(data_model, test_size=0.2, random_state=7)

## Process dependent variables

In [ ]:
target_variable = ["lamax", "laeq", "lceq", "lcpeak"]

In [ ]:
y_train = train_df[target_variable]
y_train.head(2)

In [ ]:
y_val = val_df[target_variable]
y_val.head(2)

## Process independent variables

In [ ]:
X_train = train_df.drop(target_variable + ["date"], axis=1)
X_val = val_df.drop(target_variable + ["date"], axis=1)

In [ ]:
X_train.columns

In [ ]:
one_hot_var = ["hour", "month", "weekday", "object_id", "weathercode"]
numerical_var = [col for col in X_train.columns if col not in one_hot_var]

In [ ]:
t = ColumnTransformer(
    transformers=[
        ("OneHot", OneHotEncoder(handle_unknown="ignore"), one_hot_var),
        ("StandardScaler", StandardScaler(), numerical_var),
    ]
)

# fit the encoder
t.fit(X_train)

In [ ]:
## save columnstransformer

with open("../model/model_noise_level_file42/encoder_model_file42.pkl", "wb") as file:
    pickle.dump(t, file)

In [ ]:
# Load the pickle file
with open("../model/model_noise_level_file42/encoder_model_file42.pkl", "rb") as file:
    t = pickle.load(file)

In [ ]:
# create pandas DataFrame from dense matrix
X_train = pd.DataFrame(t.fit_transform(X_train), columns=t.get_feature_names_out())

X_val = pd.DataFrame(t.transform(X_val), columns=t.get_feature_names_out())

In [ ]:
X_train.columns

In [ ]:
y_train.columns

## Predict lamax

In [ ]:
# Define the model parameters
model_params = {
    "random_forest": {
        "model": RandomForestRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "max_depth": randint(3, 50),
            "max_features": ["auto", "sqrt"],
            "min_samples_split": randint(2, 20),
            "min_samples_leaf": randint(1, 10),
            "bootstrap": [True, False],
        },
    },
    "gradient_boosting": {
        "model": GradientBoostingRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "learning_rate": uniform(0.01, 0.5),
            "max_depth": randint(1, 10),
            "min_samples_split": randint(2, 20),
            "min_samples_leaf": randint(1, 10),
        },
    },
    "xgboost": {
        "model": xgboost.XGBRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "learning_rate": uniform(0.01, 0.5),
            "max_depth": randint(1, 10),
            "min_child_weight": randint(1, 10),
            "gamma": uniform(0, 1),
            "reg_alpha": uniform(0, 1),
            "reg_lambda": uniform(0, 1),
        },
    },
}

params_dict = {}

# Loop through each model in model_params and run RandomizedSearchCV
for model_name, model_info in model_params.items():
    print("Running RandomizedSearchCV for {}...".format(model_name))

    # Create a RandomizedSearchCV object for the current model
    model = model_info["model"]
    param_dist = model_info["params"]
    random_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=1, random_state=7
    )

    # Fit the RandomizedSearchCV object to the data
    random_search.fit(X_train, y_train["lamax"])

    # Print the best parameters and score
    params_dict[model_name] = random_search.best_params_
    print("Best parameters for {}: ".format(model_name), random_search.best_params_)
    print("Best score for {}: ".format(model_name), random_search.best_score_)
    print("\n")


In [ ]:
# Save optimal param dictionary
pickle.dump(params_dict, open("../model/model_noise_level_file42/lamax_dict", "wb"))

In [ ]:
# Load param dictionary
lamax_dict = pickle.load(open("../model/model_noise_level_file42/lamax_dict", "rb"))
lamax_dict

In [ ]:
gb_params = lamax_dict["gradient_boosting"]

gb = GradientBoostingRegressor(**gb_params, random_state=7)

gb.fit(X_train, y_train["lamax"])

train_preds = gb.predict(X_train)
val_preds = gb.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lamax"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lamax"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lamax"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lamax"]))

In [ ]:
rf_params = lamax_dict["random_forest"]

rf = RandomForestRegressor(**rf_params, random_state=7)

rf.fit(X_train, y_train["lamax"])

train_preds = rf.predict(X_train)
val_preds = rf.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lamax"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lamax"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lamax"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lamax"]))

In [ ]:
xgb_params = lamax_dict["xgboost"]

xgb = xgboost.XGBRegressor(**xgb_params, random_state=7)

xgb.fit(X_train, y_train["lamax"])

train_preds = xgb.predict(X_train)
val_preds = xgb.predict(X_val)


print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lamax"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lamax"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lamax"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lamax"]))


In [ ]:
plt.scatter(val_preds, y_val["lamax"])
plt.xlabel("y pred")
plt.ylabel("y val")

In [ ]:
r2_score(val_preds, y_val["lamax"])

In [ ]:
## save model

pickle.dump(xgb, open("../model/model_noise_level_file42/xgb_lamax.pkl", "wb"))

## predict laeq

In [ ]:
# Define the model parameters
model_params = {
    "random_forest": {
        "model": RandomForestRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "max_depth": randint(3, 50),
            "max_features": ["auto", "sqrt"],
            "min_samples_split": randint(2, 20),
            "min_samples_leaf": randint(1, 10),
            "bootstrap": [True, False],
        },
    },
    "gradient_boosting": {
        "model": GradientBoostingRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "learning_rate": uniform(0.01, 0.5),
            "max_depth": randint(1, 10),
            "min_samples_split": randint(2, 20),
            "min_samples_leaf": randint(1, 10),
        },
    },
    "xgboost": {
        "model": xgboost.XGBRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "learning_rate": uniform(0.01, 0.5),
            "max_depth": randint(1, 10),
            "min_child_weight": randint(1, 10),
            "gamma": uniform(0, 1),
            "reg_alpha": uniform(0, 1),
            "reg_lambda": uniform(0, 1),
        },
    },
}

params_dict = {}

# Loop through each model in model_params and run RandomizedSearchCV
for model_name, model_info in model_params.items():
    print("Running RandomizedSearchCV for {}...".format(model_name))

    # Create a RandomizedSearchCV object for the current model
    model = model_info["model"]
    param_dist = model_info["params"]
    random_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=1, random_state=7
    )

    # Fit the RandomizedSearchCV object to the data
    random_search.fit(X_train, y_train["laeq"])

    # Print the best parameters and score
    params_dict[model_name] = random_search.best_params_
    print("Best parameters for {}: ".format(model_name), random_search.best_params_)
    print("Best score for {}: ".format(model_name), random_search.best_score_)
    print("\n")

In [ ]:
# Save optimal param dictionary
pickle.dump(params_dict, open("../model/model_noise_level_file42/laeq_dict", "wb"))

In [ ]:
# Load param dictionary
laeq_dict = pickle.load(open("../model/model_noise_level_file42/laeq_dict", "rb"))
laeq_dict

In [ ]:
gb_params = laeq_dict["gradient_boosting"]

gb = GradientBoostingRegressor(**gb_params, random_state=7)

gb.fit(X_train, y_train["laeq"])

train_preds = gb.predict(X_train)
val_preds = gb.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["laeq"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["laeq"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["laeq"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["laeq"]))

In [ ]:
rf_params = laeq_dict["random_forest"]

rf = RandomForestRegressor(**rf_params, random_state=7)

rf.fit(X_train, y_train["laeq"])

train_preds = rf.predict(X_train)
val_preds = rf.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["laeq"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["laeq"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["laeq"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["laeq"]))

In [ ]:
xgb_params = laeq_dict["xgboost"]

xgb = xgboost.XGBRegressor(**xgb_params, random_state=7)

xgb.fit(X_train, y_train["laeq"])

train_preds = xgb.predict(X_train)
val_preds = xgb.predict(X_val)


print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["laeq"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["laeq"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["laeq"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["laeq"]))

In [ ]:
plt.scatter(val_preds, y_val["laeq"])
plt.xlabel("y pred")
plt.ylabel("y val")

In [ ]:
r2_score(val_preds, y_val["laeq"])

In [ ]:
## save model

pickle.dump(xgb, open("../model/model_noise_level_file42/xgb_laeq.pkl", "wb"))

## Predict lceq

In [ ]:
# Define the model parameters
model_params = {
    "random_forest": {
        "model": RandomForestRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "max_depth": randint(3, 50),
            "max_features": ["auto", "sqrt"],
            "min_samples_split": randint(2, 20),
            "min_samples_leaf": randint(1, 10),
            "bootstrap": [True, False],
        },
    },
    "gradient_boosting": {
        "model": GradientBoostingRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "learning_rate": uniform(0.01, 0.5),
            "max_depth": randint(1, 10),
            "min_samples_split": randint(2, 20),
            "min_samples_leaf": randint(1, 10),
        },
    },
    "xgboost": {
        "model": xgboost.XGBRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "learning_rate": uniform(0.01, 0.5),
            "max_depth": randint(1, 10),
            "min_child_weight": randint(1, 10),
            "gamma": uniform(0, 1),
            "reg_alpha": uniform(0, 1),
            "reg_lambda": uniform(0, 1),
        },
    },
}

params_dict = {}

# Loop through each model in model_params and run RandomizedSearchCV
for model_name, model_info in model_params.items():
    print("Running RandomizedSearchCV for {}...".format(model_name))

    # Create a RandomizedSearchCV object for the current model
    model = model_info["model"]
    param_dist = model_info["params"]
    random_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=1, random_state=7
    )

    # Fit the RandomizedSearchCV object to the data
    random_search.fit(X_train, y_train["lceq"])

    # Print the best parameters and score
    params_dict[model_name] = random_search.best_params_
    print("Best parameters for {}: ".format(model_name), random_search.best_params_)
    print("Best score for {}: ".format(model_name), random_search.best_score_)
    print("\n")

In [ ]:
# Save optimal param dictionary
pickle.dump(params_dict, open("../model/model_noise_level_file42/lceq_dict", "wb"))

In [ ]:
# Load param dictionary
lceq_dict = pickle.load(open("../model/model_noise_level_file42/lceq_dict", "rb"))
lceq_dict

In [ ]:
gb_params = lceq_dict["gradient_boosting"]

gb = GradientBoostingRegressor(**gb_params, random_state=7)

gb.fit(X_train, y_train["lceq"])

train_preds = gb.predict(X_train)
val_preds = gb.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lceq"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lceq"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lceq"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lceq"]))

In [ ]:
rf_params = lceq_dict["random_forest"]

rf = RandomForestRegressor(**rf_params, random_state=7)

rf.fit(X_train, y_train["lceq"])

train_preds = rf.predict(X_train)
val_preds = rf.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lceq"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lceq"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lceq"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lceq"]))

In [ ]:

xgb_params = lceq_dict["xgboost"]

xgb = xgboost.XGBRegressor(**xgb_params, random_state=7)

xgb.fit(X_train, y_train["lceq"])

train_preds = xgb.predict(X_train)
val_preds = xgb.predict(X_val)


print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lceq"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lceq"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lceq"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lceq"]))

In [ ]:
plt.scatter(val_preds, y_val["lceq"])
plt.xlabel("y pred")
plt.ylabel("y val")

In [ ]:
r2_score(val_preds, y_val["lceq"])

In [ ]:
## save model

import pickle

pickle.dump(xgb, open("../model/model_noise_level_file42/xgb_lceq.pkl", "wb"))

## Predict lcpeak

In [ ]:
# Define the model parameters
model_params = {
    "random_forest": {
        "model": RandomForestRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "max_depth": randint(3, 50),
            "max_features": ["auto", "sqrt"],
            "min_samples_split": randint(2, 20),
            "min_samples_leaf": randint(1, 10),
            "bootstrap": [True, False],
        },
    },
    "gradient_boosting": {
        "model": GradientBoostingRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "learning_rate": uniform(0.01, 0.5),
            "max_depth": randint(1, 10),
            "min_samples_split": randint(2, 20),
            "min_samples_leaf": randint(1, 10),
        },
    },
    "xgboost": {
        "model": xgboost.XGBRegressor(),
        "params": {
            "n_estimators": randint(50, 100),
            "learning_rate": uniform(0.01, 0.5),
            "max_depth": randint(1, 10),
            "min_child_weight": randint(1, 10),
            "gamma": uniform(0, 1),
            "reg_alpha": uniform(0, 1),
            "reg_lambda": uniform(0, 1),
        },
    },
}

params_dict = {}

# Loop through each model in model_params and run RandomizedSearchCV
for model_name, model_info in model_params.items():
    print("Running RandomizedSearchCV for {}...".format(model_name))

    # Create a RandomizedSearchCV object for the current model
    model = model_info["model"]
    param_dist = model_info["params"]
    random_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=1, random_state=7
    )

    # Fit the RandomizedSearchCV object to the data
    random_search.fit(X_train, y_train["lcpeak"])

    # Print the best parameters and score
    params_dict[model_name] = random_search.best_params_
    print("Best parameters for {}: ".format(model_name), random_search.best_params_)
    print("Best score for {}: ".format(model_name), random_search.best_score_)
    print("\n")

In [ ]:
# Save optimal param dictionary
pickle.dump(params_dict, open("../model/model_noise_level_file42/lcpeak_dict", "wb"))

In [ ]:
# Load param dictionary
lcpeak_dict = pickle.load(open("../model/model_noise_level_file42/lcpeak_dict", "rb"))
lcpeak_dict

In [ ]:
gb_params = lcpeak_dict["gradient_boosting"]

gb = GradientBoostingRegressor(**gb_params, random_state=7)

gb.fit(X_train, y_train["lcpeak"])

train_preds = gb.predict(X_train)
val_preds = gb.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lcpeak"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lcpeak"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lcpeak"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lcpeak"]))

In [ ]:
rf_params = lcpeak_dict["random_forest"]

rf = RandomForestRegressor(**rf_params, random_state=7)

rf.fit(X_train, y_train["lcpeak"])

train_preds = rf.predict(X_train)
val_preds = rf.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lcpeak"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lcpeak"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lcpeak"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lcpeak"]))

In [ ]:
import xgboost

xgb_params = lcpeak_dict["xgboost"]

xgb = xgboost.XGBRegressor(**xgb_params, random_state=7)

xgb.fit(X_train, y_train["lcpeak"])

train_preds = xgb.predict(X_train)
val_preds = xgb.predict(X_val)


print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lcpeak"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lcpeak"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lcpeak"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lcpeak"]))

In [ ]:
plt.scatter(val_preds, y_val["lcpeak"])
plt.xlabel("y pred")
plt.ylabel("y val")

In [ ]:
r2_score(val_preds, y_val["lcpeak"])

In [ ]:
## save model

pickle.dump(xgb, open("../model/model_noise_level_file42/xgb_lcpeak.pkl", "wb"))

## In this section, we would predict lcpeak using data in file meteo


In [ ]:
meteo = pd.concat([pd.read_csv("../data/LC_data/LC_2022Q1.csv", delimiter=","),
                  pd.read_csv("../data/LC_data/LC_2022Q2.csv", delimiter=","),
                  pd.read_csv("../data/LC_data/LC_2022Q3.csv", delimiter=","),
                  pd.read_csv("../data/LC_data/LC_2022Q4.csv", delimiter=",")])

In [ ]:
meteo.columns

In [ ]:
meteo.shape

In [ ]:
meteo.head(4)

In [ ]:
meteo.dtypes

In [ ]:
meteo.isna().mean()

In [ ]:
meteo['date'] = pd.to_datetime(meteo['DATEUTC']).dt.date


In [ ]:
meteo = meteo[~meteo.LC_HUMIDITY.isnull()]

In [ ]:
meteo.isna().mean()

In [ ]:
meteo = meteo.groupby(["date", "Month", "Hour"]).mean().reset_index()

In [ ]:
meteo.shape

In [ ]:
meteo.drop(["Year", "Day", "Minute"], axis = 1, inplace = True)

In [ ]:
meteo.rename(columns={'Month': 'month',
                      'Hour': 'hour'}, inplace = True)

In [ ]:
data_model_meteo = file42.merge(meteo, how="inner",
                                left_on = ["date", "hour", "month"],
                                right_on = ["date", "hour", "month"])

In [ ]:
data_model_meteo.head(3)

In [ ]:
data_model_meteo.shape

In [ ]:
## split train, test data
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data_model_meteo, test_size=0.2, random_state = 7)

In [ ]:
target_variable = ["lamax", "laeq", "lceq", "lcpeak"]


In [ ]:
X_train = train_df.drop(target_variable + ["date"], axis = 1)
X_val = val_df.drop(target_variable + ["date"], axis = 1)

In [ ]:
X_train.columns

In [ ]:
X_train.dtypes

In [ ]:
y_train = train_df[target_variable]
y_val = val_df[target_variable]

In [ ]:
one_hot_var = ['object_id', "hour", "month", "weekday"]
numerical_var = [col for col in X_train.columns if col not in one_hot_var]

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
t = ColumnTransformer(
    transformers = [
        ('OneHot', OneHotEncoder(handle_unknown='ignore'), one_hot_var),
        ('StandardScaler', StandardScaler(), numerical_var)
        
    ])

# fit the encoder
t.fit(X_train)

In [ ]:
## save columnstransformer

with open("../model/model_noise_level_file42/encoder_model_file42_meteo.pkl", 'wb') as file:
    pickle.dump(t, file)

In [ ]:
# create pandas DataFrame from sparse matrix

from scipy.sparse import csr_matrix

X_train = pd.DataFrame.sparse.from_spmatrix(csr_matrix(t.fit_transform(X_train)))
X_train.columns  = t.get_feature_names_out()

X_val = pd.DataFrame.sparse.from_spmatrix(csr_matrix(t.fit_transform(X_val)))
X_val.columns  = t.get_feature_names_out()


In [ ]:
X_train.columns

In [ ]:
# Define the model parameters
model_params = {
    'random_forest': {
        'model': RandomForestRegressor(),
        'params': {
            'n_estimators': randint(50, 100),
            'max_depth': randint(3, 50),
            'max_features': ['auto', 'sqrt'],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'bootstrap': [True, False]
        }
    },
    'gradient_boosting': {
        'model': GradientBoostingRegressor(),
        'params': {
            'n_estimators': randint(50, 100),
            'learning_rate': uniform(0.01, 0.5),
            'max_depth': randint(1, 10),
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10)
        }
    },
    
    'xgboost': {
        'model': xgboost.XGBRegressor(),
        'params': {
            'n_estimators': randint(50, 100),
            'learning_rate': uniform(0.01, 0.5),
            'max_depth': randint(1, 10),
            'min_child_weight': randint(1, 10),
            'gamma': uniform(0, 1),
            'reg_alpha': uniform(0, 1),
            'reg_lambda': uniform(0, 1),
        }
    }
    
}

params_dict = {}

# Loop through each model in model_params and run RandomizedSearchCV
for model_name, model_info in model_params.items():
    print("Running RandomizedSearchCV for {}...".format(model_name))
    
    # Create a RandomizedSearchCV object for the current model
    model = model_info['model']
    param_dist = model_info['params']
    random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                       n_iter=10, cv=5, n_jobs=1, random_state=7)
    
    # Fit the RandomizedSearchCV object to the data
    random_search.fit(X_train, y_train['lcpeak'])
    
    # Print the best parameters and score
    params_dict[model_name] = random_search.best_params_
    print("Best parameters for {}: ".format(model_name), random_search.best_params_)
    print("Best score for {}: ".format(model_name), random_search.best_score_)
    print("\n")

In [ ]:
# Save optimal param dictionary
pickle.dump(params_dict, open("../model/model_noise_level_file42/lcpeak_dict_meteo", "wb"))

In [ ]:
# Load param dictionary
lcpeak_dict_meteo = pickle.load(open("../model/model_noise_level_file42/lcpeak_dict_meteo", "rb"))
lcpeak_dict_meteo

In [ ]:
gb_params = lcpeak_dict_meteo["gradient_boosting"]

gb = GradientBoostingRegressor(**gb_params, random_state=7)

gb.fit(X_train, y_train["lcpeak"])

train_preds = gb.predict(X_train)
val_preds = gb.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lcpeak"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lcpeak"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lcpeak"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lcpeak"]))

In [ ]:
rf_params = lcpeak_dict_meteo["random_forest"]

rf = RandomForestRegressor(**rf_params, random_state=7)

rf.fit(X_train, y_train["lcpeak"])

train_preds = rf.predict(X_train)
val_preds = rf.predict(X_val)

print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lcpeak"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lcpeak"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lcpeak"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lcpeak"]))

In [ ]:
import xgboost

xgb_params = lcpeak_dict_meteo["xgboost"]

xgb = xgboost.XGBRegressor(**xgb_params, random_state=7)

xgb.fit(X_train, y_train["lcpeak"])

train_preds = xgb.predict(X_train)
val_preds = xgb.predict(X_val)


print("Train RMSE:", np.sqrt(mean_squared_error(train_preds, y_train["lcpeak"])))
print("Val RMSE:", np.sqrt(mean_squared_error(val_preds, y_val["lcpeak"])))
print("Train MAE:", mean_absolute_error(train_preds, y_train["lcpeak"]))
print("Val MAE:", mean_absolute_error(val_preds, y_val["lcpeak"]))

In [ ]:
plt.scatter(val_preds, y_val["lcpeak"])
plt.xlabel("y pred")
plt.ylabel("y val")

In [ ]:
r2_score(val_preds, y_val["lcpeak"])

In [ ]:
## save model

pickle.dump(xgb, open("../model/model_noise_level_file42/xgb_lcpeak_meteo.pkl", "wb"))